In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

base = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
blob = PeftModel.from_pretrained(
    base,
    "workspace/bayesian-peft/checkpoints/blob_summarization/facebook/bart-base/dialoguesum/blob_summarization-dialoguesum-sample10-eps0.05-kllr0.002-beta0.2-gamma8-seed1"
)

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

ValueError: Can't find 'adapter_config.json' at 'workspace/bayesian-peft/checkpoints/blob_summarization/facebook/bart-base/dialoguesum/blob_summarization-dialoguesum-sample10-eps0.05-kllr0.002-beta0.2-gamma8-seed1'

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base.to(device).eval()
blob.to(device).eval()   # Peft/BLoB model
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

In [ ]:
from datasets import load_dataset

# DialogueSum automatically loads train/validation/test splits
ds = load_dataset("knkarthick/dialogsum")

In [ ]:
def summarize_with(model, tokenizer, dialogue, max_new_tokens=128):
    input_text = "Summarize: " + dialogue
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(ids[0], skip_special_tokens=True)

In [ ]:
for i in range(5):
    sample = ds["test"][i]
    dialogue = sample["dialogue"]
    gold = sample["summary"]

    bart_summary = summarize_with(base, tokenizer, dialogue)
    blob_summary = summarize_with(blob, tokenizer, dialogue)

    print(f"=== Example {i} ===")
    print("DIALOGUE:\n", dialogue[:400], "...\n")
    print("GOLD SUMMARY:\n", gold, "\n")
    print("--- BART-base ---")
    print(bart_summary, "\n")
    print("--- BLoB fine-tuned (LoRA) ---")
    print(blob_summary, "\n")
    print("-" * 80)